## Modeling *Ironsworn*’s core mechanic in [``dyce``](https://posita.github.io/dyce/)

Select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
try:
    import anydyce
    import dyce
    import ipywidgets
except ImportError:
    requirements = [
        "anydyce>=0.1.4,<0.2",
        "dyce>=0.5.2,<0.6",
        "ipywidgets",
    ]
    try:
        import piplite
        await piplite.install(requirements)
    except ImportError:
        import pip
        pip.main(["install"] + requirements)

In [2]:
from enum import IntEnum, auto
from dyce import H

class IronSoloResult(IntEnum):
    SPECTACULAR_FAILURE = -1
    FAILURE = auto()
    WEAK_SUCCESS = auto()
    STRONG_SUCCESS = auto()
    SPECTACULAR_SUCCESS = auto()

d6 = H(6)
d10 = H(10)

def iron_solo_dependent_term(action, first_challenge, second_challenge, mod=0):
    modded_action = action + mod
    beats_first = modded_action > first_challenge
    beats_second = modded_action > second_challenge
    doubles = first_challenge == second_challenge

    if beats_first and beats_second:
        return (
            IronSoloResult.SPECTACULAR_SUCCESS
            if doubles
            else IronSoloResult.STRONG_SUCCESS
        )
    elif beats_first or beats_second:
        return IronSoloResult.WEAK_SUCCESS
    else:
        return (
            IronSoloResult.SPECTACULAR_FAILURE
            if doubles
            else IronSoloResult.FAILURE
        )

In [ ]:
from functools import partial
from anydyce.viz import BreakoutType, jupyter_visualize

jupyter_visualize(
    [
        (f"Modifier: {mod:+}", H.foreach(
            partial(iron_solo_dependent_term, mod=mod),
            action=d6,
            first_challenge=d10,
            second_challenge=d10,
        )) for mod in range(0, 5)
    ],
    default_breakout_type=BreakoutType.BURST,
    default_main_plot_type="bar",
)